# previoius implementation

In [ ]:
def iou(a, b):
    """Compute the Intersection-Over-Union of two sets of boxes."""
    area_a = (a[:, 3] - a[:, 1]) * (a[:, 2] - a[:, 0])
    area_b = (b[:, 3] - b[:, 1]) * (b[:, 2] - b[:, 0])

    intersection_top_left = torch.max(a[:, None, :2], b[:, :2])
    intersection_bottom_right = torch.min(a[:, None, 2:], b[:, 2:])
    intersection_area = (intersection_bottom_right - intersection_top_left).clamp(min=0).prod(2)

    return intersection_area / (area_a[:, None] + area_b - intersection_area)



def get_precision(predictions, ground_truth, unknown=True):
    # Compute TP and FP for ensemble predictions of the unknown class
    if unknown is not None:
        if unknown:
            ground_truth = [gt for gt in ground_truth if gt['label_id'] == 80]
            predictions = [pred for pred in predictions if pred['label_id'] == 80]
        else: 
            ground_truth = [gt for gt in ground_truth if gt['label_id'] != 80]
            predictions = [pred for pred in predictions if pred['label_id'] != 80]
    tp, fp = compute_tp_fp(predictions, ground_truth)
    
    # Calculate precision
    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    return precision
    

def get_recall(predictions, ground_truth, iou_threshold=0.1, unknown=True):
    """Compute recall for the ensemble predictions."""
    if unknown is not None:
        if unknown:
            ground_truth = [gt for gt in ground_truth if gt['label_id'] == 80]
            predictions = [pred for pred in predictions if pred['label_id'] == 80]
        else: 
            ground_truth = [gt for gt in ground_truth if gt['label_id'] != 80]
            predictions = [pred for pred in predictions if pred['label_id'] != 80]
            
    tp, fp = compute_tp_fp(predictions, ground_truth, iou_threshold)
    fn = len(ground_truth) - tp
    
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    
    return recall


In [31]:
import os

directory_path = "output/from_dataset/filtered_unknown/SOWODB_backup/"

# List all files in the directory
all_files = os.listdir(directory_path)

# Filter the files based on the desired suffix
files = [f.split("_")[0] for f in all_files if f.endswith("t1.pth.txt")]

print(len(files))


4952


In [34]:
unknown_recall = []
known_precision = []
general_recall = []
general_precision = []

for filename in files:
    filepaths = {
    "ground_truth": f"output/from_dataset/filtered_unknown/SOWODB_backup/{filename}_gt.txt",
    "model_1": f"output/from_dataset/filtered_unknown/SOWODB_backup/{filename}_t1.pth.txt",
    }
    # Storing the data from each file in a dictionary
    data = {}
    for key, filepath in filepaths.items():
        with open(filepath, "r") as file:
            data[key] = file.readlines()
    
    h, w = map(int, data["ground_truth"][-1].strip().split(": ")[1].split(", "))
    
    # Displaying the first few lines of each file to understand the data format
    sample_data = {key: value[:5] for key, value in data.items()}
    import re
    import torch
    
    def parse_bbox(tensor_string):
        """Parse tensor string to extract bounding box coordinates."""
        coords = list(map(float, re.findall(r'([-+]?\d*\.\d+e[+-]?\d+|[-+]?\d*\.\d+|[-+]?\d+)', tensor_string)))
        return torch.tensor(coords)
    
    def parse_data_line(line):
        """Parse a line from the data files to extract bounding box, label, ID, and probability."""
        bbox = parse_bbox(re.search(r'tensor\((.*?)\)', line).group(0))
        label = re.search(r'LABEL: (.*?)(?: \(|$)', line).group(1)
        label_id = int(re.search(r'ID: (\d+)', line).group(1))
        prob = float(re. search(r'PROB: (.+)', line).group(1))
        return {'bbox': bbox, 'label': label, 'label_id': label_id, 'prob': prob}
    
    
    # Parsing the data again
    parsed_data = {key: [parse_data_line(line) for line in value[:-1]] for key, value in data.items()}
    unknown_recall.append(get_recall(parsed_data['model_1'], parsed_data['ground_truth'], unknown=True))
    known_precision.append(get_precision(parsed_data['model_1'], parsed_data['ground_truth'], unknown=False))
    # general_recall.append(get_recall(parsed_data['model_1'], parsed_data['ground_truth'], unknown=None))
    # general_precision.append(get_precision(parsed_data['model_1'], parsed_data['ground_truth'], unknown=None))


AttributeError: 'NoneType' object has no attribute 'group'

In [35]:
data

{'ground_truth': ['BBOX: tensor([114.5540, 729.9828, 362.4413, 990.8251]), LABEL: dog (ID: 8), PROB: 1.0\n',
  'BBOX: tensor([ 632.8638,  703.7109,  790.6103, 1000.2078]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([118.3099, 439.1156, 144.6009, 506.6718]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([330.5164, 576.1047, 362.4413, 681.1922]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([ 176.5258,  495.4125,  800.0000, 1182.2344]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([210.3286, 489.7828, 366.1972, 658.6734]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([202.8169, 395.9547, 292.9577, 534.8204]), LABEL: person (ID: 18), PROB: 1.0\n',
  'BBOX: tensor([561.5023, 482.2766, 584.0375, 499.1656]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([550.2347, 425.9797, 593.4272, 510.4250]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  'BBOX: tensor([582.1596, 469.1406, 629.1080, 517.9312]), LABEL: unknown (ID: 80), PROB: 1.0\n',
  

In [29]:
def mean(lst): 
    return sum(lst) / len(lst) 
 
# Driver Code 
print(mean(unknown_recall))
print(mean(known_precision))
print(mean(general_recall))
print(mean(general_precision))

0.14416783837650218
0.5043838308074982
0.36729391147968204
0.2782444016626013


# evaluation

In [70]:
import pandas as pd
ground_truth = pd.read_csv("./csv/bbox_data_scaled_gt.csv")
prob_prediction = pd.read_csv("./csv/bbox_data_scaled.csv")
custom_01_prediction = pd.read_csv("./csv/bbox_epistemic_prediction_0.01.csv")
custom_03_prediction = pd.read_csv("./csv/bbox_epistemic_prediction_0.03.csv")
custom_05_prediction = pd.read_csv("./csv/bbox_epistemic_prediction_0.05.csv")

In [86]:
gt_data = ground_truth[ground_truth['label']!='unknown']
pred_data = custom_01_prediction[custom_01_prediction['label']!='unknown']

In [87]:
import pandas as pd
import numpy as np
from collections import defaultdict

# 2. Calculate IoU
def compute_iou(box1, box2):
    x1_max, x1_min, y1_max, y1_min = box1
    x2_max, x2_min, y2_max, y2_min = box2

    intersect_min_x = max(x1_min, x2_min)
    intersect_min_y = max(y1_min, y2_min)
    intersect_max_x = min(x1_max, x2_max)
    intersect_max_y = min(y1_max, y2_max)

    intersect_area = max(0, intersect_max_x - intersect_min_x + 1) * max(0, intersect_max_y - intersect_min_y + 1)
    box1_area = (x1_max - x1_min + 1) * (y1_max - y1_min + 1)
    box2_area = (x2_max - x2_min + 1) * (y2_max - y2_min + 1)

    iou = intersect_area / float(box1_area + box2_area - intersect_area)
    return iou

# 3. Precision and Recall
tps = defaultdict(list)  # True positives per class
fps = defaultdict(list)  # False positives per class
num_gts = defaultdict(int)  # Number of ground truths per class

for _, gt in gt_data.iterrows():
    num_gts[gt['label']] += 1

for _, pred in pred_data.iterrows():
    matched = False
    for _, gt in gt_data[gt_data['img_name'] == pred['img_name']].iterrows():
        if compute_iou((gt['x_max'], gt['x_min'], gt['y_max'], gt['y_min']),
                       (pred['x_max'], pred['x_min'], pred['y_max'], pred['y_min'])) > 0.0001 \
            and gt['label'] == pred['label']:
            matched = True
            break
    if matched:
        tps[pred['label']].append(1)
        fps[pred['label']].append(0)
    else:
        tps[pred['label']].append(0)
        fps[pred['label']].append(1)

# 4. Average Precision
aps = {}  # Average precision per class

for label in tps.keys():
    tp = tps[label]
    fp = fps[label]
    tp_cumsum = np.cumsum(tp).astype(dtype=np.float32)
    fp_cumsum = np.cumsum(fp).astype(dtype=np.float32)

    recalls = tp_cumsum / num_gts[label]
    precisions = tp_cumsum / (tp_cumsum + fp_cumsum)

    # A trick to get the area under the curve for the precision-recall curve
    mrec = np.concatenate(([0.], recalls, [1.]))
    mpre = np.concatenate(([0.], precisions, [0.]))

    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = max(mpre[i - 1], mpre[i])
    idx = np.where(mrec[1:] != mrec[:-1])[0]
    aps[label] = np.sum((mrec[idx + 1] - mrec[idx]) * mpre[idx + 1])

# 5. Mean Average Precision
map_50 = np.mean(list(aps.values()))

print(f"mAP@50: {map_50}")


mAP@50: 0.29986754670427895


In [74]:
# mAP@50: 0.2958
# mAP@50: 0.29986754670427895
# mAP@50: 0.29986754670427895